# StocH

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.StocH)

class StocH(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        return x



In [3]:
model = pya.models.StocH()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'stoch'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2024
model.metadata["approved_by_author"] = '✅'
model.metadata["citation"] = "Tong, Huige, et al. \"Quantifying the stochastic component of epigenetic aging.\" Nature Aging (2024): 1-16."
model.metadata["doi"] = "https://doi.org/10.1038/s43587-024-00600-8"
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download directly with curl

In [5]:
supplementary_url = "https://figshare.com/ndownloader/files/42406308"
supplementary_file_name = "glmStocAll.Rd"
os.system(f"curl -L -o {supplementary_file_name} {supplementary_url}")

0

#### Download from R package

In [6]:
%%writefile download.r

options(repos = c(CRAN = "https://cloud.r-project.org/"))

# Function to extract and save coefficients and intercepts
ExtractCoefficients <- function(){
  load("glmStocALL.Rd")  # Load in stochastic clock information

  # Check the loaded object structure
  if (!exists("glmStocALL.lo")) {
    stop("The object glmStocALL.lo was not found in the loaded .Rd file.")
  }

  # List to store coefficients and intercepts for each clock
  coefficients_list <- list()

  for (c in 1:length(glmStocALL.lo)) {
    glm.o <- glmStocALL.lo[[c]]

    # Ensure glm.o is a glmnet object
    if (!inherits(glm.o, "glmnet")) {
      warning(paste("Object at index", c, "is not a glmnet object. Skipping."))
      next
    }

    # Extract the coefficients and intercept from the final iteration
    intercept <- glm.o$a0[length(glm.o$a0)]
    coefficients <- as.matrix(glm.o$beta)[, length(glm.o$lambda)]

    print(length(coefficients))
    print(length(rownames(coefficients)))

    # Create a data frame with feature names and coefficients
    coef_df <- data.frame(
      Feature = rownames(as.matrix(glm.o$beta)),
      Coefficient = as.numeric(coefficients),
      Intercept = rep(intercept, length(coefficients))
    )
    
    # Save each clock's coefficients to a CSV file
    write.csv(coef_df, file = paste0("Coefficients_Clock_", c, ".csv"), row.names = FALSE)
    
    # Append to the list
    coefficients_list[[c]] <- coef_df
  }

  return(coefficients_list)  # Return the list for further inspection if needed
}

# Run the function
coefficients_list <- ExtractCoefficients()

Writing download.r


In [7]:
os.system("Rscript download.r")

0

## Load features

#### From CSV file

In [8]:
df = pd.read_csv('Coefficients_Clock_1.csv')
df['feature'] = df['Feature']
df['coefficient'] = df['Coefficient']
model.features = df['feature'].tolist()

## Load weights into base model

In [9]:
weights = torch.tensor(df['coefficient'].tolist()).unsqueeze(0)
intercept = torch.tensor([df['Intercept'][0]])

#### Linear model

In [10]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [11]:
model.reference_values = None

## Load preprocess and postprocess objects

In [12]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [13]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [14]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '✅',
 'citation': 'Tong, Huige, et al. "Quantifying the stochastic component of '
             'epigenetic aging." Nature Aging (2024): 1-16.',
 'clock_name': 'stoch',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1038/s43587-024-00600-8',
 'notes': None,
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2024}
reference_values: None
preprocess_name: None
preprocess_dependencies: None
postprocess_name: None
postprocess_dependencies: None
features: ['cg00075967', 'cg00374717', 'cg00864867', 'cg00945507', 'cg01027739', 'cg01353448', 'cg01584473', 'cg01644850', 'cg01656216', 'cg01873645', 'cg01968178', 'cg02085507', 'cg02154074', 'cg02217159', 'cg02331561', 'cg02332492', 'cg02364642', 'cg02388150', 'cg02479575', 'cg02489552', 'cg02580606', 'cg02654291', 'cg02827112', 'cg02972551', 'cg03103192', 

## Basic test

In [15]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ 18.5127],
        [146.4057],
        [166.5776],
        [ 85.6306],
        [379.5047],
        [-99.5351],
        [ 42.5195],
        [234.2738],
        [213.7955],
        [ 62.3945]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [16]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [17]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: glmStocAll.Rd
Deleted file: download.r
Deleted folder: .ipynb_checkpoints
Deleted file: Coefficients_Clock_3.csv
Deleted file: Coefficients_Clock_2.csv
Deleted file: Coefficients_Clock_1.csv
